## enumerate_feature

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
# 读取已有文件
# 这里之后改成从前面直接得到 或者前面也保存成文件
dir = '../data/'
orderHistory_train = pd.read_csv(dir + 'train/orderHistory_train.csv', encoding='utf-8')
orderFuture_train = pd.read_csv(dir + 'train/orderFuture_train3.csv', encoding='utf-8')
userProfile_train = pd.read_csv(dir + 'train/userProfile_train.csv', encoding='utf-8')
userComment_train = pd.read_csv(dir + 'train/userComment_train.csv', encoding='utf-8')
action_train = pd.read_csv(dir + 'train/action_train.csv', encoding='utf-8')

orderHistory_test = pd.read_csv(dir + 'test/orderHistory_test.csv', encoding='utf-8')
orderFuture_test = pd.read_csv(dir + 'test/orderFuture_test3.csv', encoding='utf-8')
userProfile_test = pd.read_csv(dir + 'test/userProfile_test.csv', encoding='utf-8')
userComment_test = pd.read_csv(dir + 'test/userComment_test.csv', encoding='utf-8')
action_test = pd.read_csv(dir + 'test/action_test.csv', encoding='utf-8')

FileNotFoundError: [Errno 2] No such file or directory: '../data/train/orderFuture_train3.csv'

### 提取用户的行为特征：每日用户action的次数和时间

In [ ]:
start = datetime.now()
print("开始提取：", start)

# 每日用户action的次数
def user_day_count(orderFuture, action):
    temp = pd.DatetimeIndex(action['action_time'])
    action['action_date'] = temp.date
    action_date_count = action.groupby(['userid', 'action_date'])['actionType'].count().reset_index()
    action_date_count.rename(columns={'actionType': 'action_date_count'}, inplace=True)

    action_date_list = list(set(list(action.action_date)))
    column = ['userid']
    for action_date in action_date_list:
        column.append('user_day_count_' + str(action_date))
    user_date_count = pd.DataFrame(columns=column)
    user_date_count['userid'] = orderFuture['userid']

    for index, row in user_date_count.iterrows():
        print(index)
        print(row.userid)
        action_date_cnt1 = action_date_count[action_date_count['userid'] == row.userid]
        for action_date in action_date_list:
            action_date_cnt = action_date_cnt1[action_date_cnt1['action_date'] == action_date].reset_index()
            if (len(action_date_cnt) > 0):
                user_date_count['user_day_count_' + str(action_date)][user_date_count['userid'] == row.userid] = \
                action_date_cnt['action_date_count'][0]
    orderFuture = pd.merge(orderFuture, user_date_count, on='userid', how='left')
    return orderFuture

# 每日用户action的时间
def user_day_time(orderFuture, action):

    action_date_count = action.groupby(['userid', 'action_date'])['actionType_time'].sum().reset_index()
    action_date_count.rename(columns={'actionType_time': 'actionType_time_sum'}, inplace=True)
    print(action_date_count)

    action_date_list = list(set(list(action.action_date)))
    column = ['userid']
    for action_date in action_date_list:
        column.append('user_day_time_' + str(action_date))
    user_date_count = pd.DataFrame(columns=column)
    user_date_count['userid'] = orderFuture['userid']

    for index, row in user_date_count.iterrows():
        print(index)
        print(row.userid)
        action_date_cnt1 = action_date_count[action_date_count['userid'] == row.userid]
        for action_date in action_date_list:
            action_date_cnt = action_date_cnt1[action_date_cnt1['action_date'] == action_date].reset_index()
            if (len(action_date_cnt) > 0):
                user_date_count['user_day_time_' + str(action_date)][user_date_count['userid'] == row.userid] = \
                action_date_cnt['actionType_time_sum'][0]
        print(user_date_count)
    orderFuture = pd.merge(orderFuture, user_date_count, on='userid', how='left')
    return orderFuture

orderFuture_train = user_day_time(orderFuture_train, action_train)
orderFuture_test = user_day_time(orderFuture_test, action_test)

print("提取完成：", datetime.now())